In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math


def scrape_shows(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text,"html.parser")
    shows = soup.find_all("li")[2:35]
    
    shows_data = []
    for row in shows:
        shows_data.append(row.find_all("a")[0].text.strip())
    return shows_data

In [55]:
link = "https://www.ibdb.com"
showlist = scrape_shows(link)

In [56]:
showlist

['A Bronx Tale The Musical',
 'Aladdin',
 'Beautiful The Carole King Musical',
 'Cats',
 'Chicago',
 'Cirque du Soleil Paramour',
 'Dear Evan Hansen',
 'Falsettos',
 'Fiddler on the Roof',
 'Hamilton',
 'Heisenberg',
 'Holiday Inn, The New Irving Berlin Musical',
 'In Transit',
 'Jersey Boys',
 'Kinky Boots',
 'Les Liaisons Dangereuses',
 'Matilda The Musical',
 'Natasha, Pierre & The Great Comet of 1812',
 'Oh, Hello on Broadway',
 'On Your Feet!',
 'School of Rock – The Musical',
 'Something Rotten!',
 'The Book of Mormon',
 'The Cherry Orchard',
 'The Color Purple',
 'The Encounter',
 'The Front Page',
 'The Humans',
 'The Illusionists - Turn of the Century',
 'The Lion King',
 'The Phantom of the Opera',
 'Waitress',
 'Wicked']

In [77]:
import re
shows_links = {}
for show in showlist:
    name = re.sub('[^A-Za-z0-9]+', '', show).lower()
    if name[0:3] == "the":
        name = name[3:]+name[0:3]
        
    l = "https://www.allmusicals.com/{}/{}.htm".format(name[0],name)
    shows_links[show] = l

In [78]:
shows_links

{'A Bronx Tale The Musical': 'https://www.allmusicals.com/a/abronxtalethemusical.htm',
 'Aladdin': 'https://www.allmusicals.com/a/aladdin.htm',
 'Beautiful The Carole King Musical': 'https://www.allmusicals.com/b/beautifulthecarolekingmusical.htm',
 'Cats': 'https://www.allmusicals.com/c/cats.htm',
 'Chicago': 'https://www.allmusicals.com/c/chicago.htm',
 'Cirque du Soleil Paramour': 'https://www.allmusicals.com/c/cirquedusoleilparamour.htm',
 'Dear Evan Hansen': 'https://www.allmusicals.com/d/dearevanhansen.htm',
 'Falsettos': 'https://www.allmusicals.com/f/falsettos.htm',
 'Fiddler on the Roof': 'https://www.allmusicals.com/f/fiddlerontheroof.htm',
 'Hamilton': 'https://www.allmusicals.com/h/hamilton.htm',
 'Heisenberg': 'https://www.allmusicals.com/h/heisenberg.htm',
 'Holiday Inn, The New Irving Berlin Musical': 'https://www.allmusicals.com/h/holidayinnthenewirvingberlinmusical.htm',
 'In Transit': 'https://www.allmusicals.com/i/intransit.htm',
 'Jersey Boys': 'https://www.allmusic

In [79]:
shows_links["Beautiful The Carole King Musical"]='https://www.allmusicals.com/b/beautifulthecaroleking.htm'
shows_links["The Lion King"]='https://www.allmusicals.com/l/lionking.htm'
shows_links["Matilda The Musical"] = 'https://www.allmusicals.com/m/matilda.htm'
shows_links["Natasha, Pierre & The Great Comet of 1812"] = 'https://www.allmusicals.com/n/natashapierreandthegreatcometof1812.htm'
shows_links["School of Rock – The Musical"] = 'https://www.allmusicals.com/s/schoolofrock.htm'

In [2]:
def link_of_show(show_dict,show_name):
    link = show_dict[show_name]
    return link

def scrape_lyrics_links(show_dict,show_name):
    show_link = link_of_show(show_dict,show_name)
    if show_link is None:
        print(show_name +" has no lyrics")
        return False
    r = requests.get(show_link)
    soup = BeautifulSoup(r.text,"html.parser")
    
    try:
        table = soup.find("section",class_="lyrics-list")
        lyrics = table.find_all("a")
    
        lyrics_data = []
        for row in lyrics:
            lyrics_link = row["href"]
            lyrics_data.append(lyrics_link)
        return lyrics_data
    
    except (AttributeError, IndexError):
        print(show_name+" has no lyrics")
        return False
    except ValueError:
        print(show_name+" has no lyrics")
        return False

    
def scrape_lyrics_each(lyrics_link):
    try:
        r = requests.get(lyrics_link)
        soup = BeautifulSoup(r.text,"html.parser")
        table = soup.find("div", class_ = "span8 lyrics-content-main-text")
        content = table.find_all("div")[3].get_text()
        return content
    except (AttributeError, IndexError):
        return False
    except (ValueError,NameError):
        return False


def scrape_show_lyrics(show_dict,show_name):
    lyrics_links = scrape_lyrics_links(show_dict,show_name)
    if lyrics_links != False:
        filename = "{}.txt".format(show_name)
        with open(filename, "w") as f:
            for link in lyrics_links:
                if scrape_lyrics_each(link) != False:
                    content = scrape_lyrics_each(link)
                    f.write(content)
        return show_name
    else:
        return False
    
def scrape_all_shows_lyrics(show_dict):
    c = 0
    showlist = show_dict.keys()
    for show_name in showlist:
        if scrape_lyrics_links(show_dict,show_name) != False:
            scrape_show_lyrics(show_dict,show_name)
            c += 1
    print("Done!")
    print(str(c) +" shows with valid lyrics")
    

In [154]:
scrape_all_shows_lyrics(showlist)

A Bronx Tale The Musical has no lyrics
Cirque du Soleil Paramour has no lyrics
Heisenberg has no lyrics
Holiday Inn, The New Irving Berlin Musical has no lyrics
In Transit has no lyrics
Les Liaisons Dangereuses has no lyrics
Oh, Hello on Broadway has no lyrics
The Cherry Orchard has no lyrics
The Encounter has no lyrics
The Front Page has no lyrics
The Humans has no lyrics
The Illusionists - Turn of the Century has no lyrics
Done!


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [37]:
def scrape_shows_links(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text,"html.parser")
    shows = soup.find_all("a")[21:54]
    
    shows_ibdb_links={}
    for show in shows:
        name = show.text.strip()
        ibdb_link = "https://www.ibdb.com/{}".format(show["href"])
        shows_ibdb_links[name]=ibdb_link
    return shows_ibdb_links

In [38]:
link = "https://www.ibdb.com"
ibdblinks = scrape_shows_links(link)

In [ ]:
def scrape_show_info(show_name):
    link = ibdblinks[show_name]
    r = requests.get(link)
    soup = BeautifulSoup(r.text,"html.parser")
    


In [66]:
def scrape_show_songs(show_name):
    songlink = "{}/#songs".format(ibdblinks[show_name])
    r = requests.get(songlink)
    soup = BeautifulSoup(r.text,"html.parser")
    table = soup.find_all("tbody")[-1]
    tds = table.find_all("td")
    l=[]
    for td in tds:
        n = td.text.strip()
        n = n.split("\r\n")[0]
        l.append(n)
    dat = [[i,j] for i,j in zip(l[0::2],l[1::2])]
    df = pd.DataFrame(dat, columns=["ACT","Sung By"])
    return df

In [67]:
for show in showlist:
    df = scrape_show_songs(show)
    filename="{}.csv".format(show)
    df.to_csv(filename)

In [3]:
shows_url={}
for x in list(map(chr, range(97, 123))):
    link="https://www.allmusicals.com/{}.htm".format(x)
    r = requests.get(link)
    soup = BeautifulSoup(r.text,"html.parser")
    table = soup.find("div",class_="span12 sym-album-list")
    shows = table.find_all("a")

    for show in shows:
        name = show.text.strip()
        url = show["href"]
        shows_url[name] = url


In [30]:
shows_reviews_link={}
for x in shows_url.keys():
    link = shows_url[x]
    ele = link.split("/")
    e = ele[4].split(".")[0]
    l = "https://www.allmusicals.com/lyrics/{}/review.htm".format(e)
    shows_reviews_link[x]=l
    

In [31]:
shows_reviews_link

{'A Day in Hollywood / A Night in the Ukraine Lyrics': 'https://www.allmusicals.com/lyrics/adayinhollywoodanightintheukraine/review.htm',
 'A Spoof Odyssey Lyrics': 'https://www.allmusicals.com/lyrics/aspoofodyssey/review.htm',
 'Ace Lyrics': 'https://www.allmusicals.com/lyrics/ace/review.htm',
 'Across the Universe Lyrics': 'https://www.allmusicals.com/lyrics/acrosstheuniverse/review.htm',
 'Act, The Lyrics': 'https://www.allmusicals.com/lyrics/actthe/review.htm',
 'Addams Family, The Lyrics': 'https://www.allmusicals.com/lyrics/addamsfamilythe/review.htm',
 'Adding Machine Lyrics': 'https://www.allmusicals.com/lyrics/addingmachine/review.htm',
 'Adrift In Macao Lyrics': 'https://www.allmusicals.com/lyrics/adriftinmacao/review.htm',
 'Adventures of Tom Sawyer, The Lyrics': 'https://www.allmusicals.com/lyrics/adventuresoftomsawyerthe/review.htm',
 'Aida Lyrics': 'https://www.allmusicals.com/lyrics/aida/review.htm',
 "Ain't Misbehavin' Lyrics": 'https://www.allmusicals.com/lyrics/aintmi

In [36]:
import re

def scrape_all_shows_reviews(show_dict):
    c = 0
    showlist = show_dict.keys()
    for show_name in showlist:
        try:
            link = show_dict[show_name]
            r = requests.get(link)
            soup = BeautifulSoup(r.text,"html.parser")
            table = soup.find("div",class_="span8 lyrics-content-main")
            reviews = table.find_all("div")[2].get_text()
        
            filename = "{}_reviews.txt".format(re.sub('/', ' ', show_name))
            with open(filename,"w") as f:
                f.write(reviews)
            c += 1
        except (AttributeError, IndexError):
            continue
        except (ValueError,NameError):
            continue
    print("Done!")
    print(str(c) +" shows with valid review")

scrape_all_shows_reviews(shows_reviews_link)

Done!
443 shows with valid lyrics


In [40]:
shows_gross_link={}
for x in ibdblinks.keys():
    link = ibdblinks[x]
    ele = link.split("/")[-1]
    l = "https://www.ibdb.com/grosses-production/{}".format(ele)
    shows_gross_link[x]=l
   

In [41]:
shows_gross_link

{'A Bronx Tale The Musical': 'https://www.ibdb.com/grosses-production/a-bronx-tale-the-musical-508538',
 'Aladdin': 'https://www.ibdb.com/grosses-production/aladdin-495244',
 'Beautiful The Carole King Musical': 'https://www.ibdb.com/grosses-production/beautiful-the-carole-king-musical-495178',
 'Cats': 'https://www.ibdb.com/grosses-production/cats-504579',
 'Chicago': 'https://www.ibdb.com/grosses-production/chicago-4804',
 'Cirque du Soleil Paramour': 'https://www.ibdb.com/grosses-production/cirque-du-soleil-paramour-502326',
 'Dear Evan Hansen': 'https://www.ibdb.com/grosses-production/dear-evan-hansen-508238',
 'Falsettos': 'https://www.ibdb.com/grosses-production/falsettos-505384',
 'Fiddler on the Roof': 'https://www.ibdb.com/grosses-production/fiddler-on-the-roof-501123',
 'Hamilton': 'https://www.ibdb.com/grosses-production/hamilton-499521',
 'Heisenberg': 'https://www.ibdb.com/grosses-production/heisenberg-504820',
 'Holiday Inn, The New Irving Berlin Musical': 'https://www.ib

In [49]:
columns=["Date","Gross","%Gross Pot"," Attendance","%Capacity","%Previews","%Perf"]

def scrape_show_gross(show_name):
    web = shows_gross_link[show_name]
    rr = requests.get(web)
    soupp = BeautifulSoup(rr.text,"html.parser")
    
    s = soupp.find_all("div",style="float:left;margin: 2px 7px")
    
    gross_dat = []
    for i in range(len(s)):
        link = web + "/" + str(i)
        r = requests.get(link)
        soup = BeautifulSoup(r.text,"html.parser")
    
        table = soup.find("table",class_="datatable")
        tb = table.find("tbody")
        trs = tb.find_all("tr")
        for tr in trs:
            row = []
            tds = tr.find_all("td")
            for td in tds:
                row.append(td.text.strip())
            gross_dat.append(row)
    return gross_dat

In [50]:
for show in shows_gross_link.keys():
    gross_dat = scrape_show_gross(show)
    df = pd.DataFrame(gross_dat, columns=columns)
    filename="{}_gross.csv".format(show)
    df.to_csv(filename)
print("Done!")

Done!
